## General Info - 10/10/2023
The notebook was created during the lecture "Set Covering", which resulted in the template available [here](https://github.com/squillero/computational-intelligence/blob/master/2023-24/set-covering.ipynb). 

I extended the content, reorganized the code and created new functions implementing several search algorithms.

### Nomenclature
In what follows:
- a **boolean value** is a _**tile**_;
- an **array/list of booleans** is a _**set of tiles**_ or _**line of tiles**_.

#### Code Explanation

In [85]:
# import the needed libraries
from random import random
from functools import reduce
from collections import namedtuple
from itertools import count
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

To correctly model the problem, we represent a set of tiles like a boolean numpy array of size `PROBLEM_SIZE`. A tile is present or not with probability `TILE_PROBABILITY`.

A state can be modeled as a tuple of two elements:
- the first element is a set of already taken sets of tiles;
- the second element is a set of not taken sets of tiles.

> Please note that with this representation, the state tuple already contains the path.

In [86]:
PROBLEM_SIZE = 5
NUM_SETS = 10
TILE_PROBABILITY = 0.3
SETS = tuple(np.array([random() < TILE_PROBABILITY for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple("State", ["taken", "not_taken"])

The goal is reached if our taken sets of tiles can be stacked and collapsed into a single set of contiguous `PROBLEM_SIZE` tiles.

In [87]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

We can check whether the randomly generated problem is solvable by considering a solution in which we take all sets of tiles.

In [88]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

I define some functions that evaluate a state and assign a value to it based on its content.

In [89]:
def get_progressive_number(_=None, counter=count(1)):
    return next(counter)

In [90]:
def count_remaining_tiles(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [91]:
def count_number_taken_sets(state):
    return len(state.taken)

In [92]:
def sum_occupied_cells(state):
    return sum([sum(SETS[i]) for i in state.taken])

I define a function that implements the generic search algorithm. The user can specify which data structure to use as the frontier and which priority function to use.

If you use a priority queue as the frontier, the lower the priority, the earlier the state tuple will be considered for analysis.

In [93]:
def generic_search(current_state, frontier=None, priority_function=get_progressive_number):
    if not frontier:
        frontier = PriorityQueue()

    frontier.put((priority_function(current_state), current_state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
            frontier.put((priority_function(new_state), new_state))
        _, current_state = frontier.get()

    print(f"Solved in {counter:,} steps ({len(current_state.taken)} set of tiles)")
    print(f"Solution: {current_state}")

Given an initial state in which no set of tiles has been taken, we can perform different types of searches by simply changing the arguments of the `generic_search` function.

In [94]:
current_state = State(set(), set(range(NUM_SETS)))

print('-- Breadth-first search')
generic_search(current_state, SimpleQueue())
print('-- Depth-first search')
generic_search(current_state, LifoQueue())
print('-- Dijkstra with sum_occupied_cells as priority function')
generic_search(current_state, priority_function=sum_occupied_cells)
print('-- Dijkstra with count_number_taken_sets as priority function')
generic_search(current_state, priority_function=count_number_taken_sets)
print('-- Greedy Best-First search')
generic_search(current_state, priority_function=count_remaining_tiles)

-- Breadth-first search
Solved in 185 steps (3 set of tiles)
Solution: State(taken={1, 2, 6}, not_taken={0, 3, 4, 5, 7, 8, 9})
-- Depth-first search
Solved in 9 steps (9 set of tiles)
Solution: State(taken={1, 2, 3, 4, 5, 6, 7, 8, 9}, not_taken={0})
-- Dijkstra with sum_occupied_cells as priority function
Solved in 1,489 steps (5 set of tiles)
Solution: State(taken={1, 3, 4, 5, 6}, not_taken={0, 2, 7, 8, 9})
-- Dijkstra with count_number_taken_sets as priority function
Solved in 117 steps (3 set of tiles)
Solution: State(taken={1, 2, 6}, not_taken={0, 3, 4, 5, 7, 8, 9})
-- Greedy Best-First search
Solved in 4 steps (4 set of tiles)
Solution: State(taken={0, 1, 2, 6}, not_taken={3, 4, 5, 7, 8, 9})


The following function implements a depth-limited search, which by default has no limit and then performs an iterative deepening search.

In [95]:
def depth_limited_search(current_state, limit=float('inf')):
    frontier = LifoQueue()
    initial_state = current_state

    level = 0

    while level <= limit:
        frontier.put(initial_state)
        counter = 0

        while not goal_check(current_state) and frontier.qsize() > 0:
            current_state = frontier.get()
            counter += 1
            for action in current_state.not_taken:
                new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
                if len(new_state.taken) <= level:
                    frontier.put(new_state)

        if goal_check(current_state):
            break

        level += 1

    if goal_check(current_state):
        print(f"Solved in {counter:,} steps")
        print(f"Solution: {current_state}")
    else:
        print("Problem not solved. Try increasing the limit.")

In [96]:
print('-- Depth-limited search - limit: 3')
depth_limited_search(current_state, limit=3)
print('-- Depth-limited search - unbounded')
depth_limited_search(current_state)

-- Depth-limited search - limit: 3
Solved in 192 steps
Solution: State(taken={1, 6, 7}, not_taken={0, 2, 3, 4, 5, 8, 9})
-- Depth-limited search - unbounded
Solved in 192 steps
Solution: State(taken={1, 6, 7}, not_taken={0, 2, 3, 4, 5, 8, 9})


### Lab request - 19/10/2023

To implement $A^*$, we can simply modify the `generic_search` algorithm. The new function will receive as input the cost function $g(\cdot)$ and the heuristic function $h(\cdot)$. Each state $n$ will be inserted in the priority queue according to the value of $g(n) + h(n)$.

In [97]:
def a_star(current_state, cost_function=get_progressive_number, heuristic_function=get_progressive_number):
    frontier = PriorityQueue()
    frontier.put((cost_function(current_state) + heuristic_function(current_state), current_state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
            frontier.put((cost_function(new_state) + heuristic_function(new_state), new_state))
        _, current_state = frontier.get()

    print(f"Solved in {counter:,} steps ({len(current_state.taken)} set of tiles)")
    print(f"Solution: {current_state}")

In [98]:
a_star(current_state, cost_function=count_number_taken_sets, heuristic_function=count_remaining_tiles)

Solved in 4 steps (3 set of tiles)
Solution: State(taken={1, 2, 6}, not_taken={0, 3, 4, 5, 7, 8, 9})
